# Recent Load Health
Use this notebook to spot-check the most recent loads after running the ETL jobs.
Set `DATABASE_URL` before starting the kernel so results reflect your target database.


In [ ]:
import os
from datetime import datetime

import pandas as pd
from sqlalchemy import create_engine

database_url = os.environ.get("DATABASE_URL", "sqlite:///nfldb.db")
engine = create_engine(database_url)

query = """
SELECT s.year AS season,
       w.week_number AS week,
       COUNT(g.game_id) AS total_games,
       SUM(CASE WHEN g.home_points IS NOT NULL AND g.away_points IS NOT NULL THEN 1 ELSE 0 END) AS completed_games,
       MAX(g.kickoff_ts) AS last_kickoff
FROM seasons s
JOIN weeks w ON w.season_id = s.season_id
LEFT JOIN games g ON g.week_id = w.week_id
GROUP BY s.year, w.week_number
ORDER BY s.year DESC, w.week_number DESC
"""
df = pd.read_sql(query, engine)
df.head()


In [ ]:
if not df.empty:
    pivot = df.pivot(index="week", columns="season", values="completed_games")
    pivot.plot(kind="bar", figsize=(10, 4), title="Completed Games by Week")
